In [3]:
from ngsolve import *
from ngsolve.webgui import *
from netgen.occ import *

In [4]:
shape = Rectangle(1,1).Face()
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
geom = OCCGeometry(shape, dim=2)
mesh = Mesh (geom.GenerateMesh(maxh=0.05))
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [12]:
nu = Parameter(0.1)

V = VectorH1(mesh,order=2,dirichlet="bottom|right|top|left")
Q = H1(mesh,order=1);
N = NumberSpace(mesh);
X = V*Q*N
(u,p,lam), (v,q,mu) = X.TnT()
a = BilinearForm(X)
a += (nu*InnerProduct(grad(u),grad(v))+InnerProduct(grad(u)*u,v)
      -div(u)*q-div(v)*p-lam*q-mu*p)*dx

In [6]:
gfu = GridFunction(X)
#gfu.components[0].Set(CF((4*x*(1-x),0)),
#                      definedon=mesh.Boundaries("top"))
gfu.components[0].Set(CF((1,0)), definedon=mesh.Boundaries("top"))

In [7]:
from ngsolve.solvers import *

In [8]:
def SimpleNewtonSolve(gfu,a,tol=1e-13,maxits=10, callback=lambda gfu: None):
    res = gfu.vec.CreateVector()
    du = gfu.vec.CreateVector()
    fes = gfu.space
    callback(gfu)
    for it in range(maxits):
        print ("Iteration {:3}  ".format(it),end="")
        a.Apply(gfu.vec, res)
        a.AssembleLinearization(gfu.vec)
        du.data = a.mat.Inverse(fes.FreeDofs()) * res
        gfu.vec.data -= du
        callback(gfu)
        #stopping criteria
        stopcritval = sqrt(abs(InnerProduct(du,res)))
        print ("<A u",it,", A u",it,">_{-1}^0.5 = ", stopcritval)
        if stopcritval < tol:
            break

In [9]:
def SolveAndVisualize(multidim=True):
    gfu.components[0].Set(CF((4*x*(1-x),0)),
                      definedon=mesh.Boundaries("top"))
    if multidim:
        gfu_it = GridFunction(gfu.space,multidim=0)
        cb = lambda gfu : gfu_it.AddMultiDimComponent(gfu.vec) # store current state
        SimpleNewtonSolve(gfu, a, callback = cb)
    else:
        SimpleNewtonSolve(gfu, a)
    Draw(gfu.components[0],mesh, vectors = {"grid_size" : 25})
    print("above you see the solution after the Newton solve.")
    if multidim:
        Draw(gfu_it.components[0], mesh, vectors = {"grid_size" : 25})
        print("above you can inspect the results after each iteration of the Newton solve (use multidim-slider).")

In [13]:
SolveAndVisualize()

Iteration   0  <A u 0 , A u 0 >_{-1}^0.5 =  0.8821776682910251
Iteration   1  <A u 1 , A u 1 >_{-1}^0.5 =  0.025044542371290595
Iteration   2  <A u 2 , A u 2 >_{-1}^0.5 =  2.0537980208940403e-05
Iteration   3  <A u 3 , A u 3 >_{-1}^0.5 =  1.5840999939325225e-11
Iteration   4  <A u 4 , A u 4 >_{-1}^0.5 =  2.268759145142913e-16


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

above you see the solution after the Newton solve.


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

above you can inspect the results after each iteration of the Newton solve (use multidim-slider).


In [11]:
Draw(gfu.components[1],mesh, vectors = {"grid_size" : 25})

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene